In [ ]:
# !pip install SMOTE

In [ ]:
# !pip install imbalanced-learn

In [ ]:
# !pip install -U scikit-learn imbalanced-learn

In [ ]:
# pip install tensorflow

In [ ]:
# pip install xgboost

In [ ]:
# pip install gensim

In [ ]:
# pip install numpy

In [ ]:
# pip install pandas

In [ ]:
# pip install matplotlib

In [ ]:
# pip install transformers

In [ ]:
# pip install nltk

In [ ]:
# import nltk

In [ ]:
# nltk.download('stopwords')

In [ ]:
# nltk.download('punkt')

In [ ]:
# nltk.download('wordnet')

In [ ]:
pip install gensin

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
# import keras.preprocessing.text Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
import xgboost as xgb
from gensim.models import KeyedVectors
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import HashingVectorizer
# import spacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.preprocessing import Normalizer
from gensim.models import KeyedVectors
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from gensim.models import KeyedVectors
import random
import tensorflow as tf
import os

In [ ]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [ ]:
train.describe()

In [ ]:
def preprocess(text):
    pattern = r"[^\w\s]"
    text = [''.join([char if char.isalpha() else ' ' for char in word]) for word in text.split()]
    text = ' '.join(text)
    text =  re.sub(pattern," ",text)
     # used word_tokenize function to tokenize the text, gives list

    tokenized_text = word_tokenize(text.lower())
    # get the stop words
    stop_words = set(stopwords.words('english'))
    # removed stop words
    tokenized_text = [word for word in tokenized_text if word not in stop_words]
    # applying stemming
    # stemmer = PorterStemmer()
    # tokenized_text = [stemmer.stem(word) for word in tokenized_text]
    # applying lemmatization
    lemmatizer = WordNetLemmatizer()
    tokenized_text = [lemmatizer.lemmatize(word) for word in tokenized_text]

    preprocessed_text = ' '.join(tokenized_text)

    return preprocessed_text

In [ ]:
def document_vector(word2vec,doc_tokens):
  tokens = [token for token in doc_tokens if token in word2vecmodel.key_to_index]
  if not tokens:
    return np.zeros(word2vec_model.vector_size)
  doc_vector = np.mean(word2vec_model[tokens],axis=0)
  return doc_vector

In [ ]:
def reset_random_seeds(SEED=42):
   os.environ['PYTHONHASHSEED']=str(SEED)
   tf.random.set_seed(SEED)
   np.random.seed(SEED)
   random.seed(SEED)
reset_random_seeds(SEED=42)

In [ ]:
train.head(20)

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
train.duplicated().sum()

In [ ]:
train = train.drop_duplicates()

In [ ]:
train.shape

In [ ]:
class_counts = train['overall'].value_counts()

plt.bar(class_counts.index, class_counts.values)
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Histogram of Sample Counts per Class')
plt.show()

In [ ]:
test.describe()

In [ ]:
test.head(20)

In [ ]:
test.isnull().sum()

In [ ]:
test.duplicated().sum()

In [ ]:
test.shape

In [ ]:
train["Review"]= train["Review"].astype(str)


In [ ]:
train["Review"] = train["Review"].apply(preprocess)

In [ ]:
train

In [ ]:
y = train["overall"]
X_train, X_test, y_train, y_test = train_test_split(train["Review"], y, stratify=y,test_size=0.3, random_state=42)
print('Original class distribution:', Counter(y_train))


In [ ]:
X_train.shape

In [ ]:
def create_pipeline(classifiers, vectorizer, normalizer=None, reduction= False, feature_selection=True,k=1000):
     # start building the pipeline, first step -> vectorization
    step =[("vectorizer",vectorizer)]
     # if dimensionality reduction is requested add a TruncatedSVD step to the pipeline
    # if reduction:
    #     step.append(("reduction",TruncatedSVD()))
    # add normalization step
    if normalizer:
        step.append(("normalizer",normalizer))
    # if feature selection is requested add a SelectKBest step to the pipeline
    if feature_selection:
        step.append(('feature_selection', SelectKBest(chi2, k=k)))
    # add the classifier to the pipeline
    step.append(("classifier",classifiers))

        # pipeline object with the steps prepared above and assign it to the corresponding classifier name in the models dict
    models = Pipeline(step)
    return models

## Logistic Regression

### Comparing NGRAM Features

In [ ]:
class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(1,1)),k=15000)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

In [ ]:
class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(2,2)),k=20000)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

In [ ]:
class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(3,3)))
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

In [ ]:
class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(1,3)),k=24500)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

## XGBoost

In [ ]:
sampling_strategy = {i: 23000 for i in range(1, 4)}
undersampler = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train, y_train= undersampler.fit_resample(X_train, y_train)

y_train_adjusted = y_train-1
y_test = y_test-1

### NGram

In [ ]:
# space = {
#     'max_depth': hp.choice('max_depth', np.arange(3, 11, dtype=int)),
#     'n_estimators': hp.choice('n_estimators', np.arange(100, 1001, 100, dtype=int)),
#     'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
#     'subsample': hp.uniform('subsample', 0.5, 1.0),
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
#     'min_child_weight': hp.choice('min_child_weight', np.arange(1, 11, dtype=int)),
#     'reg_lambda': hp.uniform('reg_lambda', 1, 100),
#     'k': hp.choice('k', np.arange(8000, 17000, 500, dtype=int))
# }

# def objective(params):
#     k = params.pop('k')
#     feature_selector = SelectKBest(chi2, k=k)
#     X_train_sel = feature_selector.fit_transform(X_train, y_train)
#     X_test_sel = feature_selector.transform(X_test)
#     clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', **params)
#     score = cross_val_score(clf, X_train_sel, y_train_adjusted, scoring='accuracy', cv=5).mean()
#     print(f"\nTrial completed:")
#     print(f"Params: {params} features:{k}")
#     print(f"Accuracy: {score}, Loss: {-score}")
#     return {'loss': -score, 'status': STATUS_OK}
# # Run the optimization
# trials = Trials()
# best = fmin(fn=objective,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=50,
#             trials=trials)

# print("Best hyperparameters:", best)

In [ ]:
params = {'colsample_bytree': 0.8230008586447808,
 'learning_rate': 0.1909617073607323,
 'max_depth': 10,
 'min_child_weight': 4,
 'n_estimators': 1000,
 'reg_lambda': 4.163618957089685,
 'subsample': 0.59102293915828 }

classifier =  xgb.XGBClassifier(eval_metric='mlogloss', **params)

model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(1,3)),k=10500)
model.fit(X_train,y_train_adjusted)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))


### Googles Word2Vec

In [ ]:
google_word2vec = KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin", binary=True)
X_train = np.array([document_vecotr(google_word2vec,doc) for doc in X_train])
y_train = np.array([document_vecotr(google_word2vec,doc) for doc in y_train])
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))


## Random Forest Classifier

In [ ]:
classifier =  RandomForestClassifier()
model = create_pipeline(classifier, CountVectorizer(),k=10500)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

## SGD

In [ ]:
classifier =  SGDClassifier()
model = create_pipeline(classifier, CountVectorizer(),k=10500)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

## SVC

In [ ]:
classifier =  SGDClassifier()
model = create_pipeline(classifier, CountVectorizer(binary=True),k=10500)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

# Sequence Model

## Bi-Directional LSTM

## LSTM

## Transformers

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import os

In [ ]:
# # Load pre-trained model
# # 5 classes
# num_labels = 5
# model_name = "roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# if torch.cuda.is_available():
#     print(f"CUDA is available. Current device: {torch.cuda.current_device()} - {torch.cuda.get_device_name(0)}")
#     device = torch.device("cuda")
# else:
#     print("CUDA is not available, using CPU instead.")
#     device = torch.device("cpu")

# # Move model to the chosen device
# model.to(device)

In [ ]:
# df = pd.read_csv("train.csv")
# df['Review'] = df['Review'].astype(str)
# df.dropna(subset=['Review', 'overall'], inplace=True)
# df['overall'] = df['overall'] - 1

# # Split the dataset into training and testing
# train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['overall'], random_state=42)
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# # Define the tokenize function
# def tokenize_function(examples):
#     return {
#         **tokenizer(examples['Review'], truncation=True, padding="max_length", max_length=512),
#         'labels': examples['overall']
#     }

# # Tokenize the dataset
# tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=15,
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8,
#     per_device_eval_batch_size=1,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     evaluation_strategy="steps",
#     eval_steps=500,
#     save_strategy="steps",
#     save_steps=500,
#     load_best_model_at_end=True,
# )

In [ ]:
# # Initialize and train the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_test_dataset,
# )

In [ ]:
# Train the model
# trainer.train()

In [ ]:
# trainer.evaluate()

## Model Testing

### Roberta-base with 3 epochs

In [ ]:
# load the fine-tuned model and tokenizer
model_name = "bert_base_init"
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# loading test data
input_file = "test.csv"
df = pd.read_csv(input_file)
df['Review'] = df['Review'].astype(str)

# Prepare the reviews for the model
def prepare_data(reviews):
    tokenized = tokenizer(reviews, padding=True, truncation=True, max_length=512, return_tensors="pt")
    return tokenized

In [ ]:
# Classify reviews and add predictions to the DataFrame
def classify_reviews(df):
    predictions = []
    for review in df['Review']:
        inputs = prepare_data(review).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.append(preds.item() + 1) 
    df['overall'] = predictions
    return df

In [ ]:
# Apply the classification
df = classify_reviews(df)

In [ ]:
output_file = "prediction13.csv"
df[['id', 'overall']].to_csv(output_file, index=False)

### Bert-base with 3 epochs

In [ ]:
base_model =  "bert_base_init"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels = 5)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
# loading test data
input_file = "test.csv"
df = pd.read_csv(input_file)
df['Review'] = df['Review'].astype(str)

# Prepare the reviews for the model
def prepare_data(reviews):
    tokenized = tokenizer(reviews, padding=True, truncation=True, max_length=512, return_tensors="pt")
    return tokenized

# Classify reviews and add predictions to the DataFrame
def classify_reviews(df):
    predictions = []
    for review in df['Review']:
        inputs = prepare_data(review).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.append(preds.item() + 1)
    df['overall'] = predictions
    return df

In [ ]:
# Apply the classification
df = classify_reviews(df)

In [ ]:
# Save the DataFrame with predictions to a new CSV file
output_file = "./predictions/prediction17.csv"
df[['id', 'overall']].to_csv(output_file, index=False)